<a href="https://colab.research.google.com/github/matteo9903/FCC/blob/main/Huggingface%20Platform%20and%20its%20API/Text_Summarizer_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jan 28 22:50:15 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install --upgrade transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

# installo libreria transformer di huggingface con pacchetti aggiuntivi
# - mi serve download dataset da HF + installare alcune dipendenze

In [3]:
# installo e disinstallo pacchetto transfomers perché ogni tanto colab usa
# versioni vecchie

# accelerate = pacchetto utile per velocizzare operazioni con GPU e CUDA
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.48.1
Uninstalling transformers-4.48.1:
  Successfully uninstalled transformers-4.48.1
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.48.1-py3-none-any.whl (9.7 MB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)


In [4]:
# IMPORTO LIBRERIE
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
# from datasets import load_dataset, load_metric
from datasets import load_dataset, load # load_metric è versione vecchia

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# AutoModelForSeq2SeqLM serve per poter utilizzare modello LM

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# device sta usando CUDA, e dunque la GPU

'cuda'

In [7]:
# CARICO TOKENIZER per fare preprocessing e vettorizzazione
model_ckpt = "google/pegasus-cnn_dailymail"
# checkpoint tokenizer (basato su transfomers) trainato da Google
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# download del modello, deve essesre lo stesso del tokenizer!!
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# dataset HF con 16K messenger-like conversations e con sommari
dataset_samsum = load_dataset("samsum")

# posso eseguire del codice aggiuntivo già per splittare in train/test/validation

In [10]:
dataset_samsum

# ho id, dialogo esteso e sommario

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [11]:
# convesazione
dataset_samsum["train"]["dialogue"][1]


'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [12]:
# sommario della conversazione precedente
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

In [13]:
# DATA PREVIEW
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


#### Preprocessing

In [14]:
# creo funzione per seguire tokenizzazione
def convert_examples_to_features(example_batch):
    # tokenizzazione input
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    # tokenizzazione output (sommario = target)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
    # -----------------------------------------------
    # oppure (NUOVA VERSIONE) solo
    #  target_encodings = tokenizer(text_target=example_batch['summary'], padding=True, max_length = 128, truncation = True )
    # -----------------------------------------------

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [15]:
# mapping funzione sull'intero dataset
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [17]:
# rappresentazione vettoriale del primo dialogo
dataset_samsum_pt["train"]["input_ids"][1]

[18038,
 151,
 2632,
 127,
 119,
 6228,
 118,
 115,
 136,
 2974,
 152,
 10463,
 151,
 35884,
 130,
 329,
 107,
 18038,
 151,
 2587,
 314,
 1242,
 10463,
 151,
 1509,
 1]

In [18]:
# tutte le parole hanno la stessa attenzione
dataset_samsum_pt["train"]["attention_mask"][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [19]:
# label
dataset_samsum_pt["train"]["labels"][1]

[18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]

In [20]:
# Training

from transformers import DataCollatorForSeq2Seq
# ----------------------------------------------------------
# DataCollatorForSeq2Seq
# permette di caricare dati progressivamente come batch, non tutti assieme,
# per ottimizzare la gestione della memoria
# ----------------------------------------------------------

# definisco data collator
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [21]:
from transformers import TrainingArguments, Trainer

# imposto iperparametri della rete, lasciare parametri come sono a parte
# training epochs
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# inizializzo trainer
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  # train_dataset=dataset_samsum_pt["train"],
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

# ovviamente andrebbe usato il TRAINING SET, ma per ottimizzare i tempi si fa
# il training sul test set

<ipython-input-22-7a3f728c09b6>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


In [23]:
trainer.train()

# verifico training_loss che deve essere il più vicino possibile a 0

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: federer9903 (federer9903-reply). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=51, training_loss=3.0044142264945832, metrics={'train_runtime': 236.3863, 'train_samples_per_second': 3.465, 'train_steps_per_second': 0.216, 'total_flos': 313450454089728.0, 'train_loss': 3.0044142264945832, 'epoch': 0.9963369963369964})

#### Valutazione modello

Per valutare **text summarization** esistono delle apposite metriche, di cui una è il ***ROUGE SCORE***

In [24]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
      # yield = permette di ritornare valori continuamente mentre si sta eseguendo
      #         la funzione (non come return che la termina)
        yield list_of_elements[i : i + batch_size]


# funzione per valutazione modello
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [29]:
# per calcolare score devo installare libreria apposita e importare metodo "load"

!pip install evaluate

#Import evaluate library
from evaluate import load


In [30]:
# ho varie tipologie di metriche
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load('rouge')

In [37]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
    # dataset_samsum['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
    )
# ---------------------------------------------------
# in realtà punteggio si dovrebbe calcolare per tutto test set, ma qua
# si usano solo i primi 10 elementi per questioni di tempistiche
# ---------------------------------------------------

pd.DataFrame(score, index = [f'pegasus'] )

100%|██████████| 5/5 [00:21<00:00,  4.24s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023435,0.0,0.023459,0.023562


*   **più Rouge Score è vicino a 1, meglio è**

*   in questo caso è molto lontano ma perché si è effettuato training tramite test set (andrebbe fatto con **training set**) e con una sola epoca (dovrebbero essere di più)



#### Save model & Tokenizer
(modello occupa molti GB, sconsigliato salvare su Google Drive)

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
%cd /content/drive/MyDrive/CORSO_GENAI/'HUGGING FACE'/'MODEL TEXT SUMMARIZER'

/content/drive/MyDrive/CORSO_GENAI/HUGGING FACE/MODEL TEXT SUMMARIZER


In [64]:
## Save trained model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [65]:
## Save trained tokenizer
tokenizer.save_pretrained("tokenizer")
# per tokenizzare dati futuri nel caso si vogliano far analizzare dal modello

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [67]:
!pwd

/content/drive/MyDrive/CORSO_GENAI/HUGGING FACE/MODEL TEXT SUMMARIZER


In [68]:
# Load tokenizer

# do posizione del tokenizer
# tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer") # --> runtime colab
tokenizer = AutoTokenizer.from_pretrained("./tokenizer") # gdrive

In [69]:
#Prediction

# parametri predizione (cambiare solo il primo)
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
# length_penalty = serve per regolare lunghezza dell'output del model

# prendo testo e sommario del primo dialogo
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

# creo pipeline HF
pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

# dialogo originale
print("Dialogue:")
print(sample_text)

# sommario originale
print("\nReference Summary:")
print(reference)

# stampo output del modello eseguendo pipeline con i parametri scelti
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


ovviamente predizione del modello non è accurata per lo stesso motivo per cui il Rouge Score è basso